# Load Data

In [2]:
import pandas as pd
import numpy as np
from copy import deepcopy
import os

st_coords = pd.read_csv('data/seqFISH+/Out_rect_locations.csv')
st_coords.columns = ['barcode','','','x','y']
st_coords = st_coords.drop('',axis=1)
st_counts_10000genes = pd.read_csv('data/seqFISH+/Out_gene_expressions_10000genes.csv')
st_counts_10000genes = st_counts_10000genes.rename({'Unnamed: 0':'barcode'}, axis=1)
sc_counts = pd.read_csv('data/seqFISH+/raw_somatosensory_sc_exp.txt',sep='\t')
sc_counts = sc_counts.set_index('cell_id')
sc_counts = sc_counts.T
sc_labels = pd.read_csv('data/seqFISH+/somatosensory_sc_labels.txt',header=None)
sc_labels.columns = ['celltype']
celltype = list(set(sc_labels.celltype))
celltype_dict = dict(zip([x+1 for x in range(len(celltype))],celltype))
metacell_dict = dict(zip([str(x+1) for x in range(len(celltype))],celltype))
sc_labels['cluster'] = [celltype.index(x)+1 for x in sc_labels.celltype]
sc_labels['barcode'] = sc_counts.index

# load to stereoscope format

save_dir = 'data4stereoscope'
if os.path.isdir(save_dir)==False:
    os.mkdir(save_dir)

sc_counts_stereoscope = deepcopy(sc_counts)
sc_counts_stereoscope = sc_counts_stereoscope.reset_index()
sc_counts_stereoscope = sc_counts_stereoscope.rename({'index':'cell'},axis=1)
sc_counts_stereoscope.to_csv(f'{save_dir}/sc_counts.tsv',index=False,sep='\t')

sc_labels_stereoscope = deepcopy(sc_labels[['barcode','celltype']])
sc_labels_stereoscope = sc_labels_stereoscope.rename({'barcode':'cell','celltype':'bio_celltype'},axis=1)
sc_labels_stereoscope.to_csv(f'{save_dir}/sc_labels.tsv',index=False,sep='\t')

st_counts_stereoscope = deepcopy(st_counts_10000genes)
st_counts_stereoscope['index'] = st_coords.barcode
st_counts_stereoscope = st_counts_stereoscope.drop('barcode',axis=1)
st_counts_stereoscope = st_counts_stereoscope.set_index('index')
st_counts_stereoscope.to_csv(f'{save_dir}/st_coords_counts.tsv',sep='\t')

## stereoscope

In [6]:
# Run in Bash
sample=seqFISH+_10000genes
stereoscope run --sc_cnt output/data4stereoscope/${sample}/sc_counts.tsv \
--sc_labels output/data4stereoscope/${sample}/sc_labels.tsv \
--st_cnt output/data4stereoscope/${sample}/st_coords_counts.tsv \
-o output/stereoscope/${sample} -n 5000 \
--gpu -stb 1000 -scb 1000

/bin/bash: stereoscope: command not found
